In [ ]:
import sys
import os
import time

In [ ]:
# include path to our library
cwd = os.getcwd()
dir_path = os.path.dirname(os.path.realpath(cwd))
sys.path.append(dir_path)
time.sleep(2)

In [ ]:
dir_path
%cd $dir_path
!ls

In [ ]:
import argparse
import json

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import set_random_seed

import keras.backend as K
from keras.backend.tensorflow_backend import set_session

from core.data.data_loader import *
from core.models.lstm import ModelLSTM, ModelLSTMParalel, ModelLSTMCuDnnParalel
from core.models.cnn import ModelCNN
from core.models.mlp import ModelMLP
from core.models.base import BagOfHits

from core.utils.metrics import *
from core.utils.utils import *


## loading Configurartion ##


In [ ]:
# load some configurations configurations of model and others
configs = json.load(open('config_lstm.json', 'r'))
output_path = configs['paths']['save_dir']
output_logs = configs['paths']['log_dir']

data_file = configs['data']['filename']

#create a encryp name for dataset
path_to, filename = os.path.split(data_file)

orig_ds_name = filename

encryp_ds_name = get_unique_name(orig_ds_name)
decryp_ds_name = get_decryp_name(encryp_ds_name)

output_encry = os.path.join(output_path, encryp_ds_name)  
    
if os.path.isdir(output_path) == False:
    os.mkdir(output_path)

if os.path.isdir(output_encry) == False: 
    os.mkdir(output_encry)
        
if os.path.isdir(output_logs) == False:
    os.mkdir(output_logs)        

time_steps =  configs['model']['layers'][0]['input_timesteps']  # the number of points or hits
num_features = configs['model']['layers'][0]['input_features']  # the number of features of each hits
split_data = configs['data']['train_split']  # the number of features of each hits
cylindrical = configs['data']['cylindrical']  # set to polar or cartesian coordenates
#cylindrical = False
normalise = configs['data']['normalise'] 
num_hits = configs['data']['num_hits']
type_pred = configs['testing']['type_prediction']
tolerance = configs['testing']['tolerance']

    
print('OK reading of json file')


## Setup Data ##

In [ ]:
# reading path of data
path = configs['data']['filename']
print(path)
print(output_encry)

In [ ]:
data = Dataset(path, split_data, cylindrical, num_hits, KindNormalization.Zscore)
# divimos 


In [ ]:
data.data_train.iloc[0:12, 0:18]

In [ ]:
X_train, y_train = data.get_training_data(n_hit_in=time_steps, n_hit_out=1,
                                 n_features=num_features, normalise=True)
#dataset = dataset.iloc[0:2640,0:]/   b
#dataset = dataset.iloc[0:31600,0:]
print(X_train.shape)
print(y_train.shape)
X_train.head(12)

In [ ]:
y_train.head(12)

In [ ]:
def manage_models(config):
    
    type_model = config['model']['name']
    model = None

    if type_model == 'lstm': #simple LSTM
        model = ModelLSTM(config)
    elif type_model == 'lstm-paralel':
        model = ModelLSTMParalel(config)
    elif type_model == 'cnn':
        model = ModelCNN(config)
    elif type_model == 'mlp':
        model = ModelMLP(config)        

    return model


## Create Model ##

In [ ]:
# geramos o modelo
#configs = json.load(open('config_test.json', 'r'))
model = manage_models(configs)

if model is None:
    print('Please instance model')

loadModel = configs['training']['load_model']
loadModel = True

## Create Model ##

In [ ]:
from core.utils.metrics import *

# in-memory training
if loadModel == False:
    model.build_model()
    # in-memory training
    history = model.train(
        x=X_train,
        y=y_train,
        epochs = configs['training']['epochs'],
        batch_size = configs['training']['batch_size']
    )

    evaluate_training(history, output_path)

elif loadModel == True:       
    if not model.load_model():
        print ('[Error] please change the config file : load_model')

## Normal Prediction and Nearest ##
We concatenate features to predict one hit.

In [ ]:
# Prediction
import numpy as np
from core.utils.metrics import *
from core.utils.utils import *

In [ ]:
# not normalized 
x_scaler, y_scaler = data.load_scale_param(output_encry)

normalise_testing = configs['testing']['normalise']

X_test, y_test = data.get_testing_data(n_hit_in=time_steps, n_hit_out=1,
                                       n_features=num_features, normalise=normalise_testing,
                                       xscaler=x_scaler, yscaler=y_scaler)

#X_test, y_test = data.get_testing_data(n_hit_in=time_steps, n_hit_out=1,
#                                       n_features=num_features, normalise=False,
#                                       xscaler=None, yscaler=None)


#X_test = X_test.iloc[0:1000,]
#y_test = y_test[0:1000]

#orig
#data.data_test.head(12)


In [ ]:
print(X_test.shape)
print(y_test.shape)
X_test.head(6)


In [ ]:
y_test.head()


### Normal Prediction ###

In [ ]:
X_test_ = data.reshape3d(X_test, time_steps, num_features)
y_test_ = data.reshape3d(y_test, 6, num_features)

print(X_test_.shape)
print(y_test_.shape)


In [ ]:
def predict_full_sequences(x_test, data, num_hits=6, normalise=False):
    '''
        x_test: input data
        normalise: say input data must be scaled 
    '''
    timer = Timer()
    timer.start() 
    print('[Model] Predicting Sequences Started')

    total = len(x_test)

    correct = 0
    incorrect = 0
    pred_sequences = []

    for j in range(total):       
        curr_frame = x_test[j]
        predicted = []
        for i in range(num_hits):
            if normalise:
                curr_frame = data.x_scaler.transform(np.reshape(curr_frame,(1,12)))
                curr_frame_orig = data.inverse_transform_x(pd.DataFrame(curr_frame).values.flatten())
                curr_frame_orig = np.reshape(curr_frame_orig, (4,3))
                curr_frame = np.reshape(curr_frame, (4,3))
            else:
                curr_frame = curr_frame
                curr_frame_orig = curr_frame
                            
            pred = model.model.predict(curr_frame[np.newaxis,:,:])                         
            
            pred = np.reshape(pred, (1, 3))
            if normalise:
                pred = data.inverse_transform_y(pred)
            else:
                pred = pred
                
            pred = np.reshape(pred, (1, 3))
            
            predicted.append(pred)
            curr_frame = curr_frame_orig[1:]
            # inserta um  valor np.insert(array, index, value, axes)
            curr_frame = np.insert(curr_frame, [3], predicted[-1], axis=0)
            #print(curr_frame, predicted[-1])

        pred_sequences.append(predicted)

    print('[Model] Prediction Finished.')
    timer.stop()

    return pred_sequences

In [ ]:
# normalise = True say the input must be normalised
pred_full = predict_full_sequences(X_test_, data, num_hits=6, normalise=True)
print('finished prediction ...')

predicted = convert_vector_to_matrix(pred_full, 3, 6)
predicted = to_frame(predicted)
predicted.head()

In [ ]:
# original data y_true
y_test.head()

### Adding Nearest ###

In [ ]:
#bag_of_hits = None
from enum import Enum

def predict_full_sequences_nearest1(x_test, y_test, num_hits=6, num_features=3, normalise=False):
    
    timer = Timer()
    timer.start()       

    print('[Model] Predicting Sequences with Nearest Started')

    total = len(x_test)

    pred_sequences = []

    decimals = 2
    y_true = None
    
    #if normalise:
    # covert to original values
    y_true = data.inverse_transform_test_y(y_test)
        
    #y_true = y_true.round(decimals)
    
    #x_test must be in scaled form

    global bag_of_hits 
    bag_of_hits = np.array(convert_matrix_to_vec(y_true, num_features))

    y_true = np.array(y_true)
    count_correct = np.zeros(num_hits)

    for j in range(total):
        curr_frame = x_test[j]
        curr_track = y_true[j]
        
        predicted = []
        begin = 0
        #print('track :', curr_track)
        for i in range(num_hits):
            end = begin+num_features
            curr_hit_orig = curr_track[begin:end]
            begin = end
            
            #print('curr_frame:\n', curr_frame)
            # input must be scaled curr_frame
            y_pred = model.model.predict(curr_frame[np.newaxis,:,:])

            #print('pred:', y_pred)
            y_pred = np.reshape(y_pred, (1, 3))
            y_pred_orig = data.inverse_transform_y(y_pred)
            #y_pred_orig = np.round(y_pred_orig, decimals)
            #print('pred:', y_pred)
            
            print('BoH: %s, %s'%(len(bag_of_hits), bag_of_hits))

            near_pred_orig, idx = model.nearest_hit(y_pred_orig, bag_of_hits, silent=True)
            
            #print('inv pred:', y_pred_orig)
            #print('nearest:', near_pred_orig)
            #print('current:', curr_hit_orig)

            d1 = calculate_distances_vec(curr_hit_orig, y_pred_orig[0])
            d2 = calculate_distances_vec(curr_hit_orig, near_pred_orig)
            
            #print('dist pred: %s  dist near : %s ' % (d1, d2))
            
            #bag_of_hits = np.delete(bag_of_hits, idx, 0)
            
            #if np.logical_and(curr_hit, near_pred).all():
            #if np.array_equal(curr_hit, near_pred):
                          
            if d1 <= d2:
                # original coordinates
                if np.isclose(curr_hit_orig, y_pred_orig, atol=0.09).all():    
                    count_correct[i]=+1
                    
                predicted.append(y_pred)
            else:
                # original coordinates
                if np.isclose(curr_hit_orig, near_pred_orig, atol=0.09).all():    
                    count_correct[i]=+1
                               
                near_pred = data.y_scaler.transform(np.reshape(near_pred_orig, (1, 3)))
                #print('current inv:', near_pred)
                predicted.append(near_pred)
                      
            curr_frame = curr_frame[1:]
            # inserta um  valor np.insert(array, index, value, axes)
            curr_frame = np.insert(curr_frame, [3], predicted[-1], axis=0)
        
        pred_sequences.append(predicted)
        if j == 2: break
    print('[Model] Prediction Finished.')
    timer.stop()

    return pred_sequences, count_correct

def nearest_hit(hit, hits,
                    silent = True,
                    metric = 'euclidean'):

    if silent is False:
        timer = Timer()
        timer.start()

    dist = distance.cdist(hits, hit, metric)

    # get the index of minimum distance
    target_hit_index = np.argmin(dist)

    if silent is False:    
        print("--- N_hits: %s" % len(hits))
        print("--- Hit index: %s" % target_hit_index)
        print("--- " + str(metric) + " distance: " + str(dist[target_hit_index]))
        print("--- time: %s seconds" % timer.stop())

    # get hits coordinates 
    real_hit = hits[target_hit_index, :]
    real_hit = np.array(real_hit)

    # removing the hit from bag
    #hits = np.delete(hits, target_hit_index, 0)

    if dist_hit is False:
        return real_hit, target_hit_index
    else:
        return real_hit, target_hit_index, np.min(dist)

def mat_rhoetaphi_to_xyz(mat, n_hits = 5):
    pivot_tmp = 0
    for i in range(n_hits):
        pivot_tmp = i * 3
        rho = mat[pivot_tmp + 0]
        eta = mat[pivot_tmp + 1]
        phi = mat[pivot_tmp + 2]
        if (rho != 0 and eta != 0 and phi != 0):
            x, y, z = convert_rhoetaphi_to_xyz(rho, eta, phi)
            mat[pivot_tmp + 0] = x
            mat[pivot_tmp + 1] = y
            mat[pivot_tmp + 2] = z
    return mat

class BagOfHits(Enum):
    All=1,
    Track=2,
    Layer=3
    
def predict_full_sequences_nearest2(x_test, y_test, bag_of_hits, y_test_aux=None, num_hits=6, num_features=3,
                                    normalise=False, cylindrical=False, verbose=False, tol=0.01):
    
    '''
        This function shift the window by 1 new prediction each time, re-run predictions on new window
        parameters:
        x_test : X test data normaly not scaled (4 hits)
        y_test : y test data, normaly not scaled (6 hits)
        num_hits : how many hits by y_test
        normalise : it param says the input data must be scaled or not
    
    '''
    
    timer = Timer()
    timer.start()

    print('[Model] Predicting Sequences with Nearest Started')

    total = len(x_test)

    pred_sequences = []
    pred_sequences_orig = []
    
    decimals = 2
   
    # covert to original values
    #y_true = data.inverse_transform_test_y(y_test)
    #y_true = y_true.round(decimals)
    
    #change the dataset by cartesian coordinates
    if cylindrical:
        y_test = y_test_aux
    else:
        y_test = y_test
        
    # bag_of_hits  
    bag_of_hits_all = np.array(convert_matrix_to_vec(y_test, num_features))

    count_correct = np.zeros(num_hits)
    begin_idx, end_idx = 0, 0
    num_boh = 6
    
    for j in range(total):
        curr_frame = x_test[j]
        # bag_of_hit by track
        curr_track = np.array(y_test.iloc[j,0:]).reshape(num_hits, num_features)
        
        if verbose:
            print('curr_track %s , %s:' % (j , curr_track))

        predicted = []
        predicted_orig = []
        begin = 0
        for i in range(num_hits):
            # bag_of_hit by layer
            end = begin+num_features
            curr_layer = np.array(y_test.iloc[0:,begin:end]).reshape(total, num_features)
            curr_hit = curr_track[i]
            begin = end
            
            if verbose:
                # primeira esta em originais
                print('input:\n', curr_frame)
            
            if normalise:
                curr_frame = data.x_scaler.transform(np.reshape(curr_frame,(1,12)))
                curr_frame_orig = data.inverse_transform_x(pd.DataFrame(curr_frame).values.flatten())
                curr_frame_orig = np.reshape(curr_frame_orig, (4,3))
                curr_frame = np.reshape(curr_frame, (4,3))
            else:
                curr_frame = curr_frame
                curr_frame_orig = curr_frame
                
            if verbose:
                print('input:\n', curr_frame)
                #print('input orig:\n', curr_frame_orig)
            
            #print('input inv :\n', curr_frame_inv.reshape(4,3))
            #print('newaxis ', curr_frame[np.newaxis,:,:])
            # input must be scaled curr_frame
            y_pred = model.model.predict(curr_frame[np.newaxis,:,:])

            
            y_pred = np.reshape(y_pred, (1, 3))
            if normalise:
                y_pred_orig = data.inverse_transform_y(y_pred)
            else:
                y_pred_orig = y_pred

            y_pred_orig = np.reshape(y_pred_orig, (1, 3))

            if bag_of_hits == BagOfHits.All:
                hits = bag_of_hits_all
            elif bag_of_hits == BagOfHits.Track:
                hits = curr_track
            elif bag_of_hits == BagOfHits.Layer:                    
                hits = curr_layer
            
            if cylindrical:
                rho, eta, phi = y_pred_orig[0][0], y_pred_orig[0][1], y_pred_orig[0][2]
                #print(rho,eta,phi)
                x, y, z = convert_rhoetaphi_to_xyz(rho, eta, phi)
                #print(x,y,z)
                y_pred_orig[0][0] = x
                y_pred_orig[0][1] = y
                y_pred_orig[0][2] = z
                
                #print(y_pred_orig)
                    
            dist = distance.cdist(hits, y_pred_orig, 'euclidean')
            idx = np.argmin(dist)
            near_pred = hits[idx]

            # if curr_hit is in cartesian coord, near_pred must be in cartesian coord too
            if np.isclose(curr_hit, near_pred, atol=tol).all():
                count_correct[i]+=1
                #print('count_correct[%s] = %s ' % (i, count_correct[i]))
                
            if verbose:
                print('pred:', y_pred)
                print('inv pred:', y_pred_orig)
                print('current:', curr_hit)
                print('nearest:', near_pred)
            
            if cylindrical:
                x, y, z = near_pred[0], near_pred[1], near_pred[2]
                #print(x,y,z)
                rho, eta, phi = convert_xyz_to_rhoetaphi(x, y, z)
                #print(rho,eta,phi)
                near_pred[0] = rho
                near_pred[1] = eta
                near_pred[2] = phi
                
            #curr_track = np.delete(curr_track, idx, 0)
            #near_pred_orig, idx = model.nearest_hit(y_pred_orig, bag_of_hits, silent=True)
            
           
            
            '''            
            d1 = calculate_distances_vec(curr_hit, y_pred_orig[0])
            d2 = calculate_distances_vec(curr_hit, near_pred)
            #print('d1 %s, d2 %s' % (d1, d2))
            
            if d1 <= d2:
                # original coordinates
                if np.isclose(curr_hit, y_pred_orig, atol=0.09).all():    
                    count_correct[i]=+1
                #print('d1')    
                predicted.append(y_pred)
                predicted_orig.append(y_pred_orig)
            else:
                # original coordinates
                if np.isclose(curr_hit, near_pred_orig, atol=0.09).all():    
                    count_correct[i]=+1
                #print('d2')                                         
                near_pred_scaled = data.y_scaler.transform(np.reshape(near_pred, (1, 3)))
                print('near scaled:', near_pred_scaled)
                predicted.append(near_pred_scaled)
                predicted_orig.append(near_pred)
            '''
            
            '''
            near_pred = np.reshape(near_pred, (1, 3))
            if normalise:
                near_pred = data.y_scaler.transform(near_pred)
            else:
                near_pred = near_pred
            
            if verbose:
                print('near scaled:', near_pred)
            '''   
            
 
                    
            #near adiciona em valores originaeis
            predicted.append(near_pred)
            
            #predicted_orig.append(near_pred_orig)
            #print('-----\n')
            
            curr_frame = curr_frame_orig[1:]
            curr_frame = np.insert(curr_frame, [3], predicted[-1], axis=0)
            
            #curr_frame_new = data.x_scaler.transform(np.reshape(curr_frame_inv, (1, 12)))
            #curr_frame_new_inv = data.inverse_transform_x(pd.DataFrame(curr_frame).values.flatten())
            
            #print('new frame ', curr_frame_new.reshape(4,3))
            #print('new frame inv ', curr_frame_new_inv.reshape(4,3))
            
        pred_sequences.append(predicted)
        #pred_sequences_orig.append(predicted_orig)
        
        if verbose:
            if j == 3: break
                
    print('[Model] Prediction Finished.')
    timer.stop()

    return pred_sequences, count_correct

In [ ]:
#X_test = X_test.iloc[0:1000,]
#y_test = y_test[0:1000]
X_test_ = data.reshape3d(X_test, time_steps, num_features)
y_test_ = data.reshape3d(y_test, 6, num_features)

print(X_test_.shape)
print(y_test_.shape)

In [ ]:
#pred_full_res, correct = predict_full_sequences_nearest1(X_test_, y_test, 6)
#print('finished prediction nearest...')

#predicted_nearest = convert_vector_to_matrix(pred_full_res, 3, 6)
#predicted_nearest = to_frame(predicted_nearest)
#predicted_nearest.head()


In [ ]:
# aux data
data_tmp = Dataset(path, split_data, False, num_hits, KindNormalization.Zscore)
X_test_aux, y_test_aux = data_tmp.get_testing_data(n_hit_in=time_steps, n_hit_out=1,
                                 n_features=num_features, normalise=False)


In [ ]:
X_test_aux = X_test_aux.iloc[0:1000,]
y_test_aux = y_test_aux[0:1000]


In [ ]:

normalise = configs['data']['normalise'] 
# normalise = True say input data must be scaled or not. If training was normalised we must use the same.
pred_full_res, correct = model.predict_full_sequences_nearest2(X_test_, y_test, data, BagOfHits.Layer, y_test_aux=None, 6, 
                                                         normalise=normalise, cylindrical=cylindrical,
                                                         verbose=False, tol=0.01)

    
predicted_nearest = convert_vector_to_matrix(pred_full_res, 3, 6)
predicted_nearest = to_frame(predicted_nearest)
predicted_nearest.head()



In [ ]:
seq_len = num_hits - time_steps
correct = [0]
y_pred = None
if cylindrical:
    
    if type_pred == "normal":
        y_pred = model.predict_full_sequences(X_test_, data, num_hits=6, normalise=True)
    elif type_pred == "nearest":                 
        # get data in coord cartesian
        data_tmp = Dataset(data_file, split_data, False, num_hits, KindNormalization.Zscore)

        X_test_aux, y_test_aux = data_tmp.get_testing_data(n_hit_in=time_steps, n_hit_out=1,
                                         n_features=num_features, normalise=False)        
        y_pred, correct = model.predict_full_sequences_nearest(X_test_, y_test, data, BagOfHits.Layer, y_test_aux, seq_len, 
                                                             normalise=True, cylindrical=cylindrical,
                                                             verbose=False)

else:
    if type_pred == "normal":
        y_pred = model.predict_full_sequences(X_test_, data, num_hits=6, normalise=True)
    elif type_pred == "nearest": 
        y_pred, correct = model.predict_full_sequences_nearest(X_test_, y_test, data, BagOfHits.Layer, None, seq_len, 
                                                         normalise=True, cylindrical=False,
                                                         verbose=False, tol=tolerance)
    else:
        print('no algorithm defined to predict')

In [ ]:

predicted_nearest = convert_vector_to_matrix(y_pred, 3, 6)
predicted_nearest = to_frame(predicted_nearest)
predicted_nearest.head()

In [ ]:

# vetor de contagens
total = X_test.shape[0]
print(correct)
#taxa_acertos = [(correct*100)/total for x in range(0, len(correct))]
print((correct*100)/total)

In [ ]:
from matplotlib import pyplot as plt
#plt.bar(range(1,6), correct)
layers = ['layer 5', 'layer 6','layer 7','layer 8','layer 9','layer 10']

#plt.bar(np.arange(1, 6, 1), correct)
plt.bar(layers, correct)

In [ ]:
y_test.head()

In [ ]:

'''
current_track = np.array([[236.97, -107.74, -3.],
                 [332.56, -140.7, -6.6],
                 [468.9,  -177.86,  -10.2],
                 [630.02, -208.33,  -15.],
                 [791.81, -224.49,  -16.2],
                 [997.68,-224.02,  -27.]])
'''

tracks = np.array([
 [ 332.56201172, -140.69500732,   -6.5999999 ],
 [ 468.89599609, -177.86099243,  -10.19999981],
 [ 630.02398682, -208.33099365,  -15.        ],
 [ 791.80603027, -224.48699951,  -16.20000076],
 [ 997.67901611, -224.02400208,  -27.        ]])

current = [[236.97200012, -107.73600006, -3. ]]
pred = [[239.28717, -102.406906, -5.2484846]]
pred = [[ 245.10965, -117.84069,   -6.3022523]]
n = model.nearest_hit(pred, tracks, silent=False)
n

In [ ]:
near = np.array([[332.56, -140.7, -6.6 ]])
scaled = data.y_scaler.transform(near)
print(scaled)
orig = data.inverse_transform_y(scaled)
print(orig)

### Metrics for Normal Prediction ###

In [ ]:
# metricas geraeis
_,_,_,_,result = calc_score(data.reshape2d(y_test, 1),
                    data.reshape2d(predicted, 1), report=True)
print(result)

calc_score_layer(y_test, predicted, n_features=3)

mses, rmses, r2s = calc_score_layer_axes(y_test, predicted)
summarize_scores_axes(mses, rmses, r2s)


### Metrics for  Nearest Prediction ###

In [ ]:
# metricas para nearest
_,_,_,_,result = calc_score(data.reshape2d(y_test, 1),
                    data.reshape2d(predicted_nearest, 1), report=True)
print(result)

calc_score_layer(y_test, predicted_nearest, n_features=3)

mses, rmses, r2s = calc_score_layer_axes(y_test, predicted_nearest)
summarize_scores_axes(mses, rmses, r2s)

### Convert to Millimeters ###

In [ ]:
def inverse_transform_mat(scaler, data, n_features=3):
    '''
        Covert a matrix to scaled values by colum hit 
    '''
    rows = data.shape[0]
    cols = data.shape[1]
    
    begin = 0
    for i in range(0, cols, n_features):
        end = i + n_features
        hit_col = data.iloc[0:,begin:end]
        begin = end
        scaled = scaler.transform(hit_col)
        print(scaled)

inverse_transform_mat(data.y_scaler, predicted_nearest)

In [ ]:
normalise = False     
if normalise:
    print('[Data] Inverse Transforming ...')
    y_test_orig = data.inverse_transform_test_y(y_test)
    y_predicted_orig = data.inverse_transform_test_y(predicted)
    y_predicted_orig_nearest = data.inverse_transform_y(predicted_nearest)
    
    y_test_orig = pd.DataFrame(y_test_orig)    
    y_predicted_orig = pd.DataFrame(y_predicted_orig)    
else:
    y_test_orig = y_test
    y_predicted_orig = predicted
    
    y_predicted_orig_nearest = predicted_nearest
    y_predicted_orig_nearest = pd.DataFrame(y_predicted_orig_nearest)

print('[Data] shape y_predicted_orig ', y_predicted_orig.shape)    
print('[Data] shape y_test_orig ', y_test_orig.shape)

result = calc_score(data.reshape2d(y_test_orig, 1),
                    data.reshape2d(y_predicted_orig_nearest, 1), report=False)
print(result)
r2, rmse, rmses = evaluate_forecast_seq(y_test_orig, y_predicted_orig_nearest)
summarize_scores(r2, rmse, rmses)


## Prepare data for Visualization ##

In [ ]:
x_true, y_true = data.get_testing_data(n_hit_in=time_steps, n_hit_out=1,
                                 n_features=num_features, normalise=False)
x_true = x_true.iloc[0:1000,]
y_true = y_true[0:1000]

#x_true = X_test
#y_true = y_test
print(x_true.shape)
print(y_true.shape)

In [ ]:
# join both dataset must be equal to X_test
data_true = np.concatenate([x_true , y_true], axis = 1)
data_true = pd.DataFrame(data_true)
data_true.columns.name = 'real' 

print('Shape ', data_true.shape)

In [ ]:
# must be equal
#data_test.values == X_test_new.values


In [ ]:
# join both dataset should be equal to X_test but not equal
data_pred = np.concatenate([x_true , y_predicted_orig], axis = 1 )
data_pred = pd.DataFrame(data_pred)
data_pred.columns.name = 'predito'

print('Shape ', data_pred.shape)

In [ ]:
# join both dataset should be equal to X_test but not equal
data_pred_nearest = np.concatenate([x_true , y_predicted_orig_nearest], axis = 1 )
data_pred_nearest = pd.DataFrame(data_pred_nearest)

data_pred_nearest.columns.name = 'nearest'

print('Shape ', data_pred_nearest.shape)

In [ ]:
# should not be equal
#data_pred.values == X_test_new.values

In [ ]:
if cylindrical:

    y_test_orig.to_csv(os.path.join(output_path, 'y_true_%s_cylin_nearest.csv' % configs['model']['name']),
                header=False, index=False)

    y_predicted_orig.to_csv(os.path.join(output_path, 'y_pred_%s_cylin_nearest.csv' % configs['model']['name']),
                header=False, index=False)

    x_true.to_csv(os.path.join(output_path, 'x_true_%s_cylin_nearest.csv' % configs['model']['name']),
                header=False, index=False)
else:
    
    y_test_orig.to_csv(os.path.join(output_path, 'y_true_%s_xyz_nearest.csv' % configs['model']['name']),
                header=False, index=False)
    y_predicted_orig.to_csv(os.path.join(output_path, 'y_pred_%s_xyz_nearest.csv' % configs['model']['name']),
                header=False, index=False)

    x_true.to_csv(os.path.join(output_path, 'x_true_%s_xyz_nearest.csv' % configs['model']['name']),
                header=False, index=False)

print('[Output] Results saved at %', output_path)

## Visualization ##

In [ ]:
# Dados originais
N_tracks = 50
path_html = ''
name = configs['model']['name']

if cylindrical:
    path_html=os.path.join(output_path, 'track-original-%s_cylin_nearest.html' % name)
else:
    path_html=os.path.join(output_path, 'track-original-%s_xyz_nearest.html' % name)
    
fig = track_plot_xyz([data_true], n_hits = 10, cylindrical = cylindrical, n_tracks = 50, 
               path=path_html, title='Track Original #10 Hit - Model %s' % name.upper())

fig.show()

In [ ]:
# predicted data
N_tracks = 8
path_html = ''
name = configs['model']['name']

if cylindrical:
    path_html=os.path.join(output_path, 'track-predicted-%s_cylin_normal-pred.html' % name)
else:
    path_html=os.path.join(output_path, 'track-predicted-%s_xyz_normal-pred.html' % name)
    
fig = track_plot_xyz([data_true, data_pred], n_hits = 10, cylindrical = cylindrical, n_tracks = N_tracks, 
               path=path_html, title='Track Prediction #10 Hit - Model %s (Nearest hits)' % name.upper(), auto_open=True)

fig.show()

In [ ]:
# predicted data
N_tracks = 8
path_html = ''
name = configs['model']['name']

if cylindrical:
    path_html=os.path.join(output_path, 'track-predicted-%s_cylin_nearest.html' % name)
else:
    path_html=os.path.join(output_path, 'track-predicted-%s_xyz_nearest.html' % name)


fig = track_plot_xyz([data_true, data_pred_nearest], n_hits = 10, cylindrical = cylindrical, n_tracks = N_tracks, 
               path=path_html, title='Track Prediction #10 Hit - Model %s (Nearest hits)' % name.upper(), auto_open=True)

fig.show()

In [ ]:
# predicted data
N_tracks = 8
path_html = ''
name = configs['model']['name']

if cylindrical:
    path_html=os.path.join(output_path, 'track-predicted-%s_cylin_nearest.html' % name)
else:
    path_html=os.path.join(output_path, 'track-predicted-%s_xyz_nearest.html' % name)


fig = track_plot_xyz([data_true, data_pred, data_pred_nearest], n_hits = 10, cylindrical = cylindrical, n_tracks = N_tracks, 
               path=path_html, title='Track Prediction #10 Hit - Model %s (Nearest hits)' % name.upper(), auto_open=True)

fig.show()